In [55]:
import os

from langchain.llms import AI21
from langchain.prompts import PromptTemplate
from langchain.chains import SequentialChain
from langchain.callbacks import get_openai_callback
import streamlit as st


from langchain.document_loaders import PyPDFLoader

import pandas as pd
import traceback              #

from dotenv import load_dotenv
import json

In [56]:
# 0ZhUmciGlmjOZ4oGsUesOAQhWrGtpYMJ

In [57]:
# load the enviroment variable first 

load_dotenv()

True

In [58]:
# ai21 API key 
key = os.getenv("AI21_API_KEY")

In [59]:
llm = AI21( temperature = 0.8)

In [60]:
llm

AI21(temperature=0.8, ai21_api_key=SecretStr('**********'))

In [61]:
# Output format of the project

RESPONSE_JSON = {
    "1": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "2": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "3": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
}
RESPONSE_JSON

{'1': {'mcq': 'multiple choice question',
  'options': {'a': 'choice here',
   'b': 'choice here',
   'c': 'choice here',
   'd': 'choice here'},
  'correct': 'correct answer'},
 '2': {'mcq': 'multiple choice question',
  'options': {'a': 'choice here',
   'b': 'choice here',
   'c': 'choice here',
   'd': 'choice here'},
  'correct': 'correct answer'},
 '3': {'mcq': 'multiple choice question',
  'options': {'a': 'choice here',
   'b': 'choice here',
   'c': 'choice here',
   'd': 'choice here'},
  'correct': 'correct answer'}}

In [62]:
# Template making
# Here, we are giving text, subject tone for hardness level of question, Number of question

template = """
Text: {text}
You are expert MCQ maker, Given the above text. It is your job to create a QUIZ of {number} of multiple choice questions for
{subject} students in {tone}.
Make sure the question are correct and not repeated questions.
Make sure the format of your response in RESPONSE_JSON below, use it as a guide.

Ensure to make {number} of MCQ's
###RESPONSE_JSON
{response_json}

"""

In [63]:
prompt_mcq = PromptTemplate(input_variables = ['text','number','subject','tone','response_json'],
                  template = template)

In [64]:
from langchain.chains import LLMChain

generator_chain = LLMChain(llm = llm, prompt = prompt_mcq, output_key = 'quiz')

In [65]:
# template for checking the given prompt response


template2 = """
you are exper in english grammar & writer. Given the MCQ question for {subject} students. \
you need to evaluate the complexity of the MCQ question according to the MCQ grammar and complete analysi of the quiz. Only use
max 50 words for complexity analysis. If the quiz is not as per the congitive and analytical abilties of the students, update the quiz questions which needs to be changed & change the tone which needs to be changed, which perfectly fits the student
abilities. 

QUIZ MCQ : 
{quiz}

Check from an expert English Writer of the above quiz:
"""

In [66]:
quiz_evaluate_prompt = PromptTemplate(input_variables = ['subject','quiz'],
                                    template = template2)

In [67]:
review_chain = LLMChain(llm = llm, prompt = quiz_evaluate_prompt,output_key = 'review_llm_chain', verbose = True)

In [68]:
# Connecting both the chains by Seuential chain

eval_chain = SequentialChain(chains = [generator_chain, review_chain], 
                input_variables = ['text','number','subject','tone','response_json'],
                output_variables = ['quiz','review_llm_chain'])

In [69]:
with open('/Users/mohitverma/Documents/Dcoument/ML/Langchain/MCQ-genAI/sample.txt','r') as f:
    Text = f.read()

In [70]:
Text

'\nBones provide the structure for our bodies. The adult human skeleton is made up of 206 bones. These include the bones of the skull, spine (vertebrae), ribs, arms and legs. Bones are made of connective tissue reinforced with calcium and specialised bone cells. Most bones also contain bone marrow, where blood cells are made.\n\nBones work with muscles and joints to hold our body together and support freedom of movement. This is called the musculoskeletal system. The skeleton supports and shapes the body and protects delicate internal organs such as the brain, heart and lungs.\n\nBones contain most of our body’s calcium supply. The body is constantly building up and breaking down bone tissue as required. Healthy bone needs a balanced diet, regular weight-bearing exercise and the right levels of various hormones.\n\n\nThe human skeleton is made up of 206 bones, including bones of the:\n\nSkull – including the jaw bone\nSpine – cervical, thoracic and lumbar vertebrae, sacrum and tailbone

In [80]:
 
number = 3
subject = "AI"
Tone = 'Hard'
response_json = RESPONSE_JSON

In [81]:
json.dumps(response_json)

'{"1": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "2": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "3": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}}'

In [82]:
# 

eval_chain({"text": Text,
            'number': number, 'subject':subject, 'tone' : Tone, 
            'response_json' : json.dumps(response_json)} )



> Entering new LLMChain chain...
Prompt after formatting:

you are exper in english grammar & writer. Given the MCQ question for AI students. you need to evaluate the complexity of the MCQ question according to the MCQ grammar and complete analysi of the quiz. Only use
max 50 words for complexity analysis. If the quiz is not as per the congitive and analytical abilties of the students, update the quiz questions which needs to be changed & change the tone which needs to be changed, which perfectly fits the student
abilities. 

QUIZ MCQ : 
{"1": {"mcq": "The human skeleton is made up of how many bones?", "options": {"a": "300", "b": "70", "c": "206", "d": "1000"}, "correct": "c"},
{"2": {"mcq": "Short bones have a shape that is similar in length to", "options": {"a": "flat bones", "b": "long bones", "c": "irregular bones", "d": "short bones"}, "correct": "d"},
{"3": {"mcq": "The sacrum and tailbone are part of the", "options": {"a": "leg", "b": "arm", "c": "spine", "d": "skull"}, "corr

{'text': '\nBones provide the structure for our bodies. The adult human skeleton is made up of 206 bones. These include the bones of the skull, spine (vertebrae), ribs, arms and legs. Bones are made of connective tissue reinforced with calcium and specialised bone cells. Most bones also contain bone marrow, where blood cells are made.\n\nBones work with muscles and joints to hold our body together and support freedom of movement. This is called the musculoskeletal system. The skeleton supports and shapes the body and protects delicate internal organs such as the brain, heart and lungs.\n\nBones contain most of our body’s calcium supply. The body is constantly building up and breaking down bone tissue as required. Healthy bone needs a balanced diet, regular weight-bearing exercise and the right levels of various hormones.\n\n\nThe human skeleton is made up of 206 bones, including bones of the:\n\nSkull – including the jaw bone\nSpine – cervical, thoracic and lumbar vertebrae, sacrum and

In [84]:
# The output is in string, we need this json

response = eval_chain({"text": Text,
            'number': number, 'subject':subject, 'tone' : 'Extremely hard', 
                    'response_json' : json.dumps(response_json)} )



> Entering new LLMChain chain...
Prompt after formatting:

you are exper in english grammar & writer. Given the MCQ question for AI students. you need to evaluate the complexity of the MCQ question according to the MCQ grammar and complete analysi of the quiz. Only use
max 50 words for complexity analysis. If the quiz is not as per the congitive and analytical abilties of the students, update the quiz questions which needs to be changed & change the tone which needs to be changed, which perfectly fits the student
abilities. 

QUIZ MCQ : 
{
"1": {
"mcq": "The human skeleton contains how many bones?",
"options": {
"a": "0",
"b": "100",
"c": "206",
"d": "1"
},
"correct": "206"
},
"2": {
"mcq": "What type of bone contains bone marrow?",
"options": {
"a": "long",
"b": "short",
"c": "flat",
"d": "irregular"
},
"correct": "irregular"
},
"3": {
"mcq": "What are the four main functions of bones?",
"options": {
"a": "support",
"b": "protection",
"c": "movement",
"d": "storage"
},
"correct": "s

In [85]:
quiz1 = response.get('quiz')


In [86]:
print(quiz1)

{
"1": {
"mcq": "The human skeleton contains how many bones?",
"options": {
"a": "0",
"b": "100",
"c": "206",
"d": "1"
},
"correct": "206"
},
"2": {
"mcq": "What type of bone contains bone marrow?",
"options": {
"a": "long",
"b": "short",
"c": "flat",
"d": "irregular"
},
"correct": "irregular"
},
"3": {
"mcq": "What are the four main functions of bones?",
"options": {
"a": "support",
"b": "protection",
"c": "movement",
"d": "storage"
},
"correct": "support, protection, movement and storage"
}
}


In [87]:
#!pip install simplejson

In [92]:
import simplejson as json


quiz1 = json.loads(quiz1)
quiz1

{'1': {'mcq': 'The human skeleton contains how many bones?',
  'options': {'a': '0', 'b': '100', 'c': '206', 'd': '1'},
  'correct': '206'},
 '2': {'mcq': 'What type of bone contains bone marrow?',
  'options': {'a': 'long', 'b': 'short', 'c': 'flat', 'd': 'irregular'},
  'correct': 'irregular'},
 '3': {'mcq': 'What are the four main functions of bones?',
  'options': {'a': 'support',
   'b': 'protection',
   'c': 'movement',
   'd': 'storage'},
  'correct': 'support, protection, movement and storage'}}

In [89]:
print(quiz.strip())

{
"1": {
"mcq": "What is the part of the human skeleton that includes the ribs?",
"options": {"a": "spine", "b": "skull", "c": "chest", "d": "arms"},
"correct": "chest"
},
"2": {
"mcq": "What type of bone is the wrist made up of?",
"options": {"a": "long bone", "b": "short bone", "c": "flat bone", "d": "irregular bone"},
"correct": "short bone"
},
"3": {
"mcq": "Which of the following bones make up the human spine?",
"options": {"a": "cervical vertebrae", "b": "lumbar vertebrae", "c": "thoracic vertebrae", "d": "sacrum", "e": "coccyx"},
"correct": "cervical, thoracic and lumbar vertebrae, sacrum and tailbone (coccyx)"
},
"4": {
"mcq": "In the human body, which of the following bones contains bone marrow?",
"options": {


In [94]:
table_data = []

for key, val in quiz1.items():
    mcq = val['mcq']
    
    options = "|".join(
        [
            f"{option} : {option_value}"
            for option, option_value in val['options'].items()
        ]
      )
    
    correct = val['correct']
    table_data.append({"MCQ": mcq, "Choices": options, "Correct": correct})

In [95]:
table_data

[{'MCQ': 'The human skeleton contains how many bones?',
  'Choices': 'a : 0|b : 100|c : 206|d : 1',
  'Correct': '206'},
 {'MCQ': 'What type of bone contains bone marrow?',
  'Choices': 'a : long|b : short|c : flat|d : irregular',
  'Correct': 'irregular'},
 {'MCQ': 'What are the four main functions of bones?',
  'Choices': 'a : support|b : protection|c : movement|d : storage',
  'Correct': 'support, protection, movement and storage'}]

In [96]:
quiz2 = pd.DataFrame(table_data)

In [97]:
quiz2.to_csv("quiz.csv",index=False)

In [98]:
pd.read_csv("quiz.csv")

,MCQ,Choices,Correct
0,The human skeleton contains how many bones?,a : 0|b : 100|c : 206|d : 1,206
1,What type of bone contains bone marrow?,a : long|b : short|c : flat|d : irregular,irregular
2,What are the four main functions of bones?,a : support|b : protection|c : movement|d : st...,"support, protection, movement and storage"
